In [1]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ningom","key":"53b8540e06ccbede8759befaff246b35"}'}

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!mv kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [8]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 53.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.20MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 26.4MB/s]


In [0]:
#データ取得
import pandas as pd
import numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [0]:
#drop dates
train_df.drop('PassengerId', axis=1, inplace=True)
train_df.drop('Name', axis=1, inplace=True)
train_df.drop('Ticket', axis=1, inplace=True)
train_df.drop('Cabin', axis=1, inplace=True)
#drop dates
test_df.drop('PassengerId', axis=1, inplace=True)
test_df.drop('Name', axis=1, inplace=True)
test_df.drop('Ticket', axis=1, inplace=True)
test_df.drop('Cabin', axis=1, inplace=True)

In [0]:
# 文字列の数値置換
train_df.Embarked = train_df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
train_df.Sex = train_df.Sex.replace(['male', 'female'], [0, 1])
train_df.Age = train_df.Age.replace('NaN', 0)
test_df.Embarked = test_df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
test_df.Sex = test_df.Sex.replace(['male', 'female'], [0, 1])
test_df.Age = test_df.Age.replace('NaN', 0)

In [0]:
#データの分割(t_train:正解ラベル,x_train:トレーニングデータ)
t_train=train_df["Survived"].values
x_train=train_df
x_train.drop("Survived",axis=1,inplace=True)
x_train=x_train.values

In [0]:
#データの分割
x_test=test_df.values

In [0]:
#正規化
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std_scl = MinMaxScaler() #MinMaxScaler() or Robustscaler
x_train=std_scl.fit_transform(x_train).astype(np.int64)
x_test=std_scl.fit_transform(x_test).astype(np.int64)

In [18]:
#モデルの構築,学習 
from sklearn.cluster import KMeans
model=KMeans(n_clusters=2)
model.fit(x_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [0]:
#K-means結果の格納 
y_train=model.predict(x_train)

In [22]:
#結果を検証
from sklearn.metrics import accuracy_score

acc_train=accuracy_score(y_train,t_train)
print(acc_train)

0.5342312008978676
